In [13]:
import pandas as pd
import datetime
from datetime import date, timedelta
import numpy as np
import seaborn as sns

In [14]:
def missing_date(d, start_date, end_date):
    #print(start_date, end_date)
    date_set = set(start_date + timedelta(x) for x in range((end_date - start_date).days))
    #print(d)
    missing = sorted(date_set - set(d))
    #print(missing)
    return missing

def toDate(row):
    return datetime.datetime.strptime(row['date'], "%Y-%m-%d").date()

def duplicate_rows(x, colnm):
    return x[colnm == x.duplicated([colnm])]

def get_outliers_index(x,colnm):
    # print("Processing ",x.iloc[0]['store_id'], x.iloc[0]['cat_id'])
    data = x[colnm]
    Q1 = np.percentile(data, 25, interpolation = 'midpoint') 
    Q2 = np.percentile(data, 50, interpolation = 'midpoint') 
    Q3 = np.percentile(data, 75, interpolation = 'midpoint') 
    
    # print('Q1 25 percentile of the given data is, ', Q1)
    # print('Q1 50 percentile of the given data is, ', Q2)
    # print('Q1 75 percentile of the given data is, ', Q3)
    
    IQR = Q3 - Q1 
    # print('Interquartile range is', IQR)
    low_lim = Q1 - 1.5 * IQR
    up_lim = Q3 + 1.5 * IQR
    # print('low_limit is', low_lim)
    # print('up_limit is', up_lim)
    df = x
    idx = df.index[(df[colnm] > up_lim) | (df[colnm] < low_lim)]
    # print(x.loc[idx])
    # print("\n*************************\n")
    return idx #x.loc[idx]

def print_outliers(x,colnm):
    idx = get_outliers_index(x,colnm)
    return x.loc[idx]

def set_outliers_to_nan(x,colnm,mndta):
    idx = get_outliers_index(x,colnm)
    #x.loc[idx,[colnm]] = None
    mndta.loc[idx,[colnm]] = None
    return mndta.loc[idx]
    

### Read the data sales.csv from the data folder into a pandas DataFrame and preview the first five rows.

In [15]:
data = pd.read_csv("/Zia/workflow_files/data/sales.csv")

In [16]:
data

,date,store_id,cat_id,sales
0,2011-01-29,TX_1,FOODS,3950.35
1,2011-01-30,TX_1,FOODS,3844.97
2,2011-01-31,TX_1,FOODS,2888.03
3,2011-02-01,TX_1,FOODS,3631.28
4,2011-02-02,TX_1,FOODS,3072.18
...,...,...,...,...
58225,2016-05-18,WI_3,HOUSEHOLD,2225.31
58226,2016-05-19,WI_3,HOUSEHOLD,2468.50
58227,2016-05-20,WI_3,HOUSEHOLD,3664.25
58228,2016-05-21,WI_3,HOUSEHOLD,3487.02


In [17]:
sd_df = data.copy()

In [18]:
sd_df

,date,store_id,cat_id,sales
0,2011-01-29,TX_1,FOODS,3950.35
1,2011-01-30,TX_1,FOODS,3844.97
2,2011-01-31,TX_1,FOODS,2888.03
3,2011-02-01,TX_1,FOODS,3631.28
4,2011-02-02,TX_1,FOODS,3072.18
...,...,...,...,...
58225,2016-05-18,WI_3,HOUSEHOLD,2225.31
58226,2016-05-19,WI_3,HOUSEHOLD,2468.50
58227,2016-05-20,WI_3,HOUSEHOLD,3664.25
58228,2016-05-21,WI_3,HOUSEHOLD,3487.02


### Verify the number of unique values of store_id and cat_id.

In [19]:
print("Number of unique values of 'store_id': ",sd_df['store_id'].nunique())
print("Number of unique values of 'cat_id': ",sd_df['cat_id'].nunique())

Number of unique values of 'store_id':  10
Number of unique values of 'cat_id':  3


### Verify the date range of the sales data.

In [20]:
# sd_df['rdate'] =  sd_df.apply(toDate, axis=1)
sd_df['rdate'] =  pd.to_datetime(sd_df['date'])

#df['race_label'] = df.apply(label_race, axis=1)
sd_df['rdate']

0       2011-01-29
1       2011-01-30
2       2011-01-31
3       2011-02-01
4       2011-02-02
           ...    
58225   2016-05-18
58226   2016-05-19
58227   2016-05-20
58228   2016-05-21
58229   2016-05-22
Name: rdate, Length: 58230, dtype: datetime64[ns]

In [21]:
dtr = sd_df['rdate'].max() - sd_df['rdate'].min()
print("Date range of sales data : ", dtr)

Date range of sales data :  1940 days 00:00:00


In [22]:
sd_df['rdate'].nunique()

1941

In [23]:
shp_cat_grp = sd_df.groupby(['store_id','cat_id'])
shp_cat_grp

### Check for duplicated dates within a store and category group

In [24]:
r = shp_cat_grp.apply(lambda x: duplicate_rows(x,'rdate'))
r

C:\Users\Ziauddin Syed\AppData\Local\Temp\ipykernel_14776\2257613844.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  r = shp_cat_grp.apply(lambda x: duplicate_rows(x,'rdate'))


,,,date,store_id,cat_id,sales,rdate
store_id,cat_id,,,,,,


In [25]:
for name, group in shp_cat_grp:
	print(name, type(group))
	print("duplicate dates in ", name ,group['rdate' == group.duplicated(['rdate'])])
	print("\n")

('CA_1', 'FOODS') <class 'pandas.core.frame.DataFrame'>
duplicate dates in  ('CA_1', 'FOODS') Empty DataFrame
Columns: [date, store_id, cat_id, sales, rdate]
Index: []


('CA_1', 'HOBBIES') <class 'pandas.core.frame.DataFrame'>
duplicate dates in  ('CA_1', 'HOBBIES') Empty DataFrame
Columns: [date, store_id, cat_id, sales, rdate]
Index: []


('CA_1', 'HOUSEHOLD') <class 'pandas.core.frame.DataFrame'>
duplicate dates in  ('CA_1', 'HOUSEHOLD') Empty DataFrame
Columns: [date, store_id, cat_id, sales, rdate]
Index: []


('CA_2', 'FOODS') <class 'pandas.core.frame.DataFrame'>
duplicate dates in  ('CA_2', 'FOODS') Empty DataFrame
Columns: [date, store_id, cat_id, sales, rdate]
Index: []


('CA_2', 'HOBBIES') <class 'pandas.core.frame.DataFrame'>
duplicate dates in  ('CA_2', 'HOBBIES') Empty DataFrame
Columns: [date, store_id, cat_id, sales, rdate]
Index: []


('CA_2', 'HOUSEHOLD') <class 'pandas.core.frame.DataFrame'>
duplicate dates in  ('CA_2', 'HOUSEHOLD') Empty DataFrame
Columns: [date, 

### Check if there are any missing dates in between the sales data.

In [26]:
max_dt = max(sd_df['rdate'])
max_dt

Timestamp('2016-05-22 00:00:00')

In [27]:
min_dt = min(sd_df['rdate'])
min_dt

Timestamp('2011-01-29 00:00:00')

In [28]:
shp_cat_grp[['rdate']].apply(lambda x: missing_date(x['rdate'],min_dt,max_dt))

store_id  cat_id   
CA_1      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
CA_2      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
CA_3      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
CA_4      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
TX_1      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
TX_2      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
TX_3      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
WI_1      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
WI_2      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
WI_3      FOODS        []
          HOBBIES      []
          HOUSEHOLD    []
dtype: object

### Check for outliers by using the IQR method and implement it on the sales data.

In [29]:
z = shp_cat_grp.apply(lambda x: print_outliers(x,'sales'))
z

C:\Users\Ziauddin Syed\AppData\Local\Temp\ipykernel_14776\2656595661.py:39: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  idx = get_outliers_index(x,colnm)
C:\Users\Ziauddin Syed\AppData\Local\Temp\ipykernel_14776\1327513869.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  z = shp_cat_grp.apply(lambda x: print_outliers(x,'sales'))


date store_id     cat_id     sales      rdate
store_id cat_id                                                              
CA_1     FOODS     17799  2011-12-25     CA_1      FOODS      0.00 2011-12-25
                   18165  2012-12-25     CA_1      FOODS      0.00 2012-12-25
                   18530  2013-12-25     CA_1      FOODS      0.00 2013-12-25
                   18660  2014-05-04     CA_1      FOODS  12279.28 2014-05-04
                   18814  2014-10-05     CA_1      FOODS  11763.78 2014-10-05
...                              ...      ...        ...       ...        ...
WI_3     HOUSEHOLD 58178  2016-04-01     WI_3  HOUSEHOLD   4256.42 2016-04-01
                   58179  2016-04-02     WI_3  HOUSEHOLD   4200.40 2016-04-02
                   58180  2016-04-03     WI_3  HOUSEHOLD   4281.67 2016-04-03
                   58187  2016-04-10     WI_3  HOUSEHOLD   4151.65 2016-04-10
                   58207  2016-04-30     WI_3  HOUSEHOLD   4077.24 2016-04-30

[828 rows x 5 columns]

### Check for any missing sales values.

In [30]:
sd_df[sd_df['sales'].isnull()]

,date,store_id,cat_id,sales,rdate


### Set outliers to null value and impute the missing sales using the linear interpolation method.

#### Set outliers to null

In [31]:
y = shp_cat_grp.apply(lambda x: set_outliers_to_nan(x,'sales',sd_df))
y

C:\Users\Ziauddin Syed\AppData\Local\Temp\ipykernel_14776\2656595661.py:43: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  idx = get_outliers_index(x,colnm)
C:\Users\Ziauddin Syed\AppData\Local\Temp\ipykernel_14776\2656595661.py:43: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  idx = get_outliers_index(x,colnm)
C:\Users\Ziauddin Syed\AppData\Local\Temp\ipykernel_14776\2656595661.py:43: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encou

date store_id     cat_id  sales      rdate
store_id cat_id                                                           
CA_1     FOODS     17799  2011-12-25     CA_1      FOODS    NaN 2011-12-25
                   18165  2012-12-25     CA_1      FOODS    NaN 2012-12-25
                   18530  2013-12-25     CA_1      FOODS    NaN 2013-12-25
                   18660  2014-05-04     CA_1      FOODS    NaN 2014-05-04
                   18814  2014-10-05     CA_1      FOODS    NaN 2014-10-05
...                              ...      ...        ...    ...        ...
WI_3     HOUSEHOLD 58178  2016-04-01     WI_3  HOUSEHOLD    NaN 2016-04-01
                   58179  2016-04-02     WI_3  HOUSEHOLD    NaN 2016-04-02
                   58180  2016-04-03     WI_3  HOUSEHOLD    NaN 2016-04-03
                   58187  2016-04-10     WI_3  HOUSEHOLD    NaN 2016-04-10
                   58207  2016-04-30     WI_3  HOUSEHOLD    NaN 2016-04-30

[828 rows x 5 columns]

In [32]:
sd_df[sd_df['sales'].isnull()]

,date,store_id,cat_id,sales,rdate
330,2011-12-25,TX_1,FOODS,NaN,2011-12-25
434,2012-04-07,TX_1,FOODS,NaN,2012-04-07
435,2012-04-08,TX_1,FOODS,NaN,2012-04-08
463,2012-05-06,TX_1,FOODS,NaN,2012-05-06
533,2012-07-15,TX_1,FOODS,NaN,2012-07-15
...,...,...,...,...,...
58178,2016-04-01,WI_3,HOUSEHOLD,NaN,2016-04-01
58179,2016-04-02,WI_3,HOUSEHOLD,NaN,2016-04-02
58180,2016-04-03,WI_3,HOUSEHOLD,NaN,2016-04-03
58187,2016-04-10,WI_3,HOUSEHOLD,NaN,2016-04-10


In [34]:
nlidx = sd_df[sd_df['sales'].isnull()].index

In [35]:
list(nlidx)

[330,
 434,
 435,
 463,
 533,
 696,
 960,
 981,
 988,
 1061,
 1135,
 1170,
 1177,
 1408,
 1426,
 1506,
 1527,
 1590,
 1598,
 1716,
 1791,
 1884,
 1933,
 1940,
 2271,
 2637,
 3002,
 3367,
 3412,
 3431,
 3432,
 3433,
 3440,
 3454,
 3461,
 3475,
 3481,
 3489,
 3509,
 3510,
 3517,
 3524,
 3539,
 3597,
 3692,
 3732,
 3733,
 3776,
 3790,
 3797,
 3853,
 3866,
 3874,
 3880,
 4212,
 4578,
 4943,
 5308,
 5388,
 5409,
 5450,
 5451,
 5465,
 5549,
 5673,
 5674,
 5731,
 5738,
 5745,
 5759,
 5773,
 5801,
 5815,
 5822,
 6153,
 6251,
 6257,
 6286,
 6519,
 6748,
 6783,
 6811,
 6884,
 6975,
 7048,
 7249,
 7338,
 7339,
 7421,
 7614,
 7756,
 8094,
 8460,
 8825,
 9190,
 9256,
 9270,
 9279,
 9280,
 9284,
 9319,
 9332,
 9362,
 9555,
 9682,
 9683,
 9697,
 9704,
 10035,
 10401,
 10455,
 10766,
 11131,
 11197,
 11220,
 11221,
 11371,
 11449,
 11496,
 11497,
 11539,
 11546,
 11553,
 11554,
 11561,
 11568,
 11609,
 11610,
 11638,
 11645,
 11976,
 12342,
 12707,
 13072,
 13244,
 13271,
 13397,
 13437,
 13481,
 1356

####  Impute the missing sales using the linear interpolation method

In [38]:
sd_df['sales'] = sd_df['sales'].interpolate()

#### Check the interploted values

In [39]:
sd_df.iloc[nlidx]

,date,store_id,cat_id,sales,rdate
330,2011-12-25,TX_1,FOODS,4965.8100,2011-12-25
434,2012-04-07,TX_1,FOODS,5980.2600,2012-04-07
435,2012-04-08,TX_1,FOODS,5428.8400,2012-04-08
463,2012-05-06,TX_1,FOODS,6503.9850,2012-05-06
533,2012-07-15,TX_1,FOODS,5223.8000,2012-07-15
...,...,...,...,...,...
58178,2016-04-01,WI_3,HOUSEHOLD,2586.8525,2016-04-01
58179,2016-04-02,WI_3,HOUSEHOLD,2637.7050,2016-04-02
58180,2016-04-03,WI_3,HOUSEHOLD,2688.5575,2016-04-03
58187,2016-04-10,WI_3,HOUSEHOLD,3359.3850,2016-04-10


### Save the preprocessed data as 'sales_processed.csv'

In [41]:
sd_df.to_csv('sales_processed.csv')